<a href="https://colab.research.google.com/github/srivastavavansh94/Sentimenet-Analyzer-using-BERT/blob/main/BERT_on_Custom_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.model_selection import train_test_split

In [ ]:
df1 = pd.read_csv('train.csv')
df2 = pd.read_csv('test.csv')

In [ ]:
df = pd.concat([df1, df2.head(15000)], ignore_index=True)

In [ ]:
print(df.tail())

                                                    text sentiment
39995  Reading through most of the other reviews, I t...       neg
39996  Like many western Pennsylvania history buffs, ...       neg
39997  For all its wonderful images, for all of its g...       neg
39998  Hadn't really heard too much about this movie ...       neg
39999  This is definitely an excellent show. I don't ...       pos


In [ ]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['sentiment'] = label_encoder.fit_transform(df['sentiment'])  # Assuming train_df is your DataFrame

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

In [ ]:
train_dataset

Dataset({
    features: ['text', 'sentiment', '__index_level_0__'],
    num_rows: 32000
})

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def preprocess_function(examples):
    model_inputs = tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)
    model_inputs['labels'] = examples['sentiment']  # Assuming sentiment is already in label form (0 or 1)
    return model_inputs

In [ ]:
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_val_dataset = val_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/32000 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

In [ ]:

tokenized_train_dataset

Dataset({
    features: ['text', 'sentiment', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 32000
})

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,  # Use the validation dataset here
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
1,0.188300,0.188920


TrainOutput(global_step=2000, training_loss=0.23438112258911134, metrics={'train_runtime': 3190.6188, 'train_samples_per_second': 10.029, 'train_steps_per_second': 0.627, 'total_flos': 8419553771520000.0, 'train_loss': 0.23438112258911134, 'epoch': 1.0})

In [ ]:
evaluation_results = trainer.evaluate()
print(evaluation_results)

{'eval_loss': 0.1889197677373886, 'eval_runtime': 233.7197, 'eval_samples_per_second': 34.229, 'eval_steps_per_second': 2.139, 'epoch': 1.0}


In [ ]:
model.save_pretrained("/content/drive/MyDrive/fine-tuned-custom-sentiment-model")
tokenizer.save_pretrained("/content/drive/MyDrive/fine-tuned-custom-sentiment-model")

('/content/drive/MyDrive/fine-tuned-custom-sentiment-model/tokenizer_config.json',
 '/content/drive/MyDrive/fine-tuned-custom-sentiment-model/special_tokens_map.json',
 '/content/drive/MyDrive/fine-tuned-custom-sentiment-model/vocab.txt',
 '/content/drive/MyDrive/fine-tuned-custom-sentiment-model/added_tokens.json')

In [ ]:
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=512)
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = logits.argmax(dim=-1).item()  # Get the predicted class index
    return predicted_class

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

In [ ]:
tokenizer = BertTokenizer.from_pretrained("/content/drive/MyDrive/fine-tuned-custom-sentiment-model")
model = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/fine-tuned-custom-sentiment-model")

In [ ]:
new_review = "The show was amazing! I loved every minute of it."
predicted_sentiment = predict_sentiment(new_review)
print(f"Predicted sentiment for review '{new_review}': {predicted_sentiment}")
# 1 => Positive   0 => Negative

Predicted sentiment for review 'The show was amazing! I loved every minute of it.': 1


In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model.push_to_hub("custom-sentiment-model")
tokenizer.push_to_hub("custom-sentiment-model")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/vansh02062002/custom-sentiment-model/commit/6c22e4d5919309e2ab96d9643329e680ceb51608', commit_message='Upload tokenizer', commit_description='', oid='6c22e4d5919309e2ab96d9643329e680ceb51608', pr_url=None, pr_revision=None, pr_num=None)